In [1]:
import pandas as pd
import subprocess
import os
from typing import Dict, Set, List, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def get_hapmap_positions(hapmap_file: str) -> Dict[int, Set[str]]:
    """Read HapMap3 SNP positions by chromosome"""
    print("Reading HapMap3 positions...")
    hapmap_df = pd.read_csv(hapmap_file, sep='\s+', header=None,
                           names=['SNP', 'chromosome', 'morgans', 'position', 'ref', 'alt'])
    
    positions_by_chr = {}
    for chrom in range(1, 23):
        chr_data = hapmap_df[hapmap_df['chromosome'] == chrom]
        positions_by_chr[chrom] = set(f"{chrom}:{pos}" for pos in chr_data['position'])
    
    return positions_by_chr

def get_1000g_positions(vcf_file: str) -> Set[str]:
    """Extract SNP positions from VCF file"""
    cmd = f"bcftools query -f '%CHROM:%POS\n' {vcf_file}"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return set(result.stdout.strip().split('\n'))

def filter_vcf_by_positions(input_vcf: str, output_vcf: str, positions: Set[str]):
    """Filter VCF file to keep only specified positions"""
    # Create temporary positions file
    temp_pos_file = f"temp_positions_{os.path.basename(input_vcf)}.txt"
    with open(temp_pos_file, 'w') as f:
        for pos in positions:
            chrom, pos = pos.split(':')
            f.write(f"{chrom}\t{pos}\n")
    
    # Filter VCF
    cmd = f'bcftools view -T {temp_pos_file} -Oz -o {output_vcf} {input_vcf}'
    subprocess.run(cmd, shell=True, check=True)
    
    # Index filtered file
    subprocess.run(f'bcftools index {output_vcf}', shell=True, check=True)
    
    # Remove temporary file
    os.remove(temp_pos_file)

def process_chromosome(chrom: int, hapmap_positions: Set[str], 
                      data_dir: str, output_dir: str) -> Tuple[int, Dict]:
    """Process a single chromosome to find and filter common SNPs"""
    try:
        # Input files
        vcf_file = os.path.join(data_dir, f'filtered.chr{chrom}.phase3.vcf.gz')
        
        # Get 1000G positions
        kg_positions = get_1000g_positions(vcf_file)
        
        # Find common positions
        common_positions = hapmap_positions.intersection(kg_positions)
        
        # Create output directory if needed
        os.makedirs(output_dir, exist_ok=True)
        
        # Filter VCF file
        output_vcf = os.path.join(output_dir, f'common_snps.chr{chrom}.vcf.gz')
        filter_vcf_by_positions(vcf_file, output_vcf, common_positions)
        
        stats = {
            'chromosome': chrom,
            'hapmap_count': len(hapmap_positions),
            'kg_count': len(kg_positions),
            'common_count': len(common_positions)
        }
        
        return chrom, stats
        
    except Exception as e:
        print(f"Error processing chromosome {chrom}: {e}")
        return chrom, None

def main():
    # Configuration
    DATA_DIR = '../data/1000g'
    OUTPUT_DIR = '../data/common_snps'
    HAPMAP_FILE = '/Users/jingl1/Desktop/CMU/02704/hw_data/02704_data/HapMap3.snp'  # Update with your HapMap3 file path
    
    # Create output directory
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # Get HapMap positions by chromosome
    hapmap_positions = get_hapmap_positions(HAPMAP_FILE)
    
    # Process all chromosomes
    stats_all = []
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_chrom = {}
        for chrom in range(1, 23):
            future = executor.submit(
                process_chromosome,
                chrom,
                hapmap_positions[chrom],
                DATA_DIR,
                OUTPUT_DIR
            )
            future_to_chrom[future] = chrom
        
        for future in tqdm(as_completed(future_to_chrom), total=22):
            chrom = future_to_chrom[future]
            _, stats = future.result()
            if stats:
                stats_all.append(stats)
    
    # Create summary
    stats_df = pd.DataFrame(stats_all)
    stats_df['hapmap_pct'] = (stats_df['common_count'] / stats_df['hapmap_count'] * 100).round(2)
    stats_df['kg_pct'] = (stats_df['common_count'] / stats_df['kg_count'] * 100).round(2)
    
    # Save summary
    summary_file = os.path.join(OUTPUT_DIR, 'common_snps_summary.csv')
    stats_df.to_csv(summary_file, index=False)
    
    # Print summary
    print("\nSummary of common SNPs:")
    print(f"Total HapMap SNPs: {stats_df['hapmap_count'].sum():,}")
    print(f"Total 1000G SNPs: {stats_df['kg_count'].sum():,}")
    print(f"Total common SNPs: {stats_df['common_count'].sum():,}")
    print(f"\nDetailed statistics saved to: {summary_file}")

if __name__ == "__main__":
    main()

Reading HapMap3 positions...


100%|███████████████████████████████████████████| 22/22 [05:49<00:00, 15.89s/it]


Summary of common SNPs:
Total HapMap SNPs: 697,393
Total 1000G SNPs: 81,192,381
Total common SNPs: 21,643

Detailed statistics saved to: ../data/common_snps/common_snps_summary.csv


In [6]:
import os
import subprocess
import pandas as pd
import numpy as np
from tqdm import tqdm
import tempfile

class VCFPopulationConverter:
    def __init__(self, data_dir: str = '../data/1000g', common_snps_dir: str = '../data/common_snps'):
        self.data_dir = data_dir
        self.common_snps_dir = common_snps_dir
        self.output_dir = os.path.join(data_dir, 'populations')
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Load population mapping
        self.sample_pop_map = self._load_population_mapping()
    
    def _load_population_mapping(self):
        panel_file = os.path.join(self.data_dir, 'integrated_call_samples.panel')
        panel_df = pd.read_csv(panel_file, sep='\t')
        return dict(zip(panel_df['sample'], panel_df['pop']))
    
    def _get_population_samples(self, population):
        samples = [sample for sample, pop in self.sample_pop_map.items() 
                  if pop == population]
        return sorted(samples)
    
    def _convert_chromosome(self, chrom, population, samples):
        """Convert a single chromosome for a population"""
        print(f"Processing chromosome {chrom} for {population}...")
        
        # Input VCF file
        vcf_file = os.path.join(self.common_snps_dir, f'common_snps.chr{chrom}.vcf.gz')
        
        # Create temporary directory
        with tempfile.TemporaryDirectory() as temp_dir:
            # Create sample file
            sample_file = os.path.join(temp_dir, 'samples.txt')
            with open(sample_file, 'w') as f:
                for sample in samples:
                    f.write(f"{sample}\n")
            
            # Extract population VCF
            temp_vcf = os.path.join(temp_dir, f'temp_{population}.vcf.gz')
            cmd = f"bcftools view -S {sample_file} -Oz -o {temp_vcf} {vcf_file}"
            subprocess.run(cmd, shell=True, check=True)
            subprocess.run(f"bcftools index {temp_vcf}", shell=True, check=True)
            
            # Get variant info
            variant_file = os.path.join(temp_dir, 'variants.txt')
            cmd = f"bcftools query -f '%CHROM\t%POS\t%REF\t%ALT\n' {temp_vcf} > {variant_file}"
            subprocess.run(cmd, shell=True, check=True)
            
            variant_data = pd.read_csv(variant_file, sep='\t',
                                     names=['chromosome', 'position', 'ref', 'alt'])
            n_variants = len(variant_data)
            
            # Get and convert genotypes
            geno_file = os.path.join(temp_dir, 'genotypes.txt')
            cmd = f"bcftools query -f '%CHROM\t%POS[\t%GT]\n' {temp_vcf} > {geno_file}"
            subprocess.run(cmd, shell=True, check=True)
            
            # Read and convert genotypes
            n_samples = len(samples)
            geno_array = np.zeros((n_variants, n_samples), dtype=np.uint8)
            
            print(f"Converting genotypes for {n_variants} variants and {n_samples} samples...")
            with open(geno_file, 'r') as f:
                for i, line in enumerate(tqdm(f, total=n_variants)):
                    parts = line.strip().split('\t')
                    gts = parts[2:]  # Skip CHROM and POS
                    
                    for j, gt in enumerate(gts):
                        if gt in ['0|0', '0/0']:
                            code = 0
                        elif gt in ['0|1', '1|0', '0/1', '1/0']:
                            code = 1
                        elif gt in ['1|1', '1/1']:
                            code = 2
                        else:
                            code = 9
                        geno_array[i, j] = code
            
            # Verify genotype conversion
            unique_gts = np.unique(geno_array, return_counts=True)
            print("\nGenotype distribution:")
            for gt, count in zip(*unique_gts):
                print(f"Genotype {gt}: {count} ({count/(n_variants*n_samples)*100:.2f}%)")
        
        return geno_array, variant_data
    
    def convert_population(self, population):
        """Convert all chromosomes for a population to HapMap format"""
        print(f"Converting data for population: {population}")
        
        # Get samples
        samples = self._get_population_samples(population)
        if not samples:
            raise ValueError(f"No samples found for population {population}")
        
        print(f"Found {len(samples)} samples for {population}")
        
        # Process chromosomes
        all_genotypes = []
        all_variants = []
        
        for chrom in range(1, 23):
            geno_array, variant_data = self._convert_chromosome(chrom, population, samples)
            all_genotypes.append(geno_array)
            all_variants.append(variant_data)
        
        # Combine data
        final_genotypes = np.vstack(all_genotypes)
        final_variants = pd.concat(all_variants, ignore_index=True)
        
        # Save output files
        output_prefix = os.path.join(self.output_dir, population)
        
        # 1. SNP file
        final_variants['morgans'] = 0.0
        snp_file = f"{output_prefix}.snp"
        final_variants.to_csv(snp_file, sep='\t', index=False, header=False)
        
        # 2. Sample file
        ind_data = pd.DataFrame({
            'sample': samples,
            'sex': 'U',
            'pop': population
        })
        ind_file = f"{output_prefix}.ind"
        ind_data.to_csv(ind_file, sep='\t', index=False, header=False)
        
        # 3. Genotype file
        geno_file = f"{output_prefix}.geno"
        print(f"\nWriting genotypes to {geno_file}...")
        with open(geno_file, 'w') as f:
            for row in tqdm(final_genotypes):
                f.write(''.join(map(str, row)) + '\n')
        
        print(f"\nConversion complete for {population}!")
        print(f"Files created:")
        print(f"1. {snp_file} - SNP information ({len(final_variants)} variants)")
        print(f"2. {ind_file} - Sample information ({len(samples)} samples)")
        print(f"3. {geno_file} - Genotype data")
        
        # Print final statistics
        total_gts = final_genotypes.size
        gt_counts = np.unique(final_genotypes, return_counts=True)
        print("\nFinal genotype distribution:")
        for gt, count in zip(*gt_counts):
            print(f"Genotype {gt}: {count} ({count/total_gts*100:.2f}%)")
        
        return final_genotypes, final_variants, samples

    def convert_all_populations(self, populations=None):
        if populations is None:
            populations = ['ASW', 'CEU', 'CHB', 'JPT', 'LWK', 'MXL', 'TSI', 'YRI']
        
        for population in populations:
            try:
                print(f"\nProcessing population: {population}")
                self.convert_population(population)
            except Exception as e:
                print(f"Error processing population {population}: {e}")

if __name__ == "__main__":
    converter = VCFPopulationConverter()
    converter.convert_all_populations()


Processing population: ASW
Converting data for population: ASW
Found 61 samples for ASW
Processing chromosome 1 for ASW...
Converting genotypes for 1617 variants and 61 samples...


100%|███████████████████████████████████| 1617/1617 [00:00<00:00, 145872.36it/s]



Genotype distribution:
Genotype 0: 92256 (93.53%)
Genotype 1: 3976 (4.03%)
Genotype 2: 2354 (2.39%)
Genotype 9: 51 (0.05%)
Processing chromosome 2 for ASW...
Converting genotypes for 1786 variants and 61 samples...


100%|███████████████████████████████████| 1786/1786 [00:00<00:00, 142331.08it/s]



Genotype distribution:
Genotype 0: 102249 (93.85%)
Genotype 1: 4523 (4.15%)
Genotype 2: 2055 (1.89%)
Genotype 9: 119 (0.11%)
Processing chromosome 3 for ASW...
Converting genotypes for 1446 variants and 61 samples...


...


Genotype distribution:
Genotype 0: 34555 (94.10%)
Genotype 1: 1518 (4.13%)
Genotype 2: 639 (1.74%)
Genotype 9: 8 (0.02%)
Processing chromosome 22 for YRI...
Converting genotypes for 286 variants and 108 samples...



100%|██████████████████████████████████████| 286/286 [00:00<00:00, 75302.63it/s]



Genotype distribution:
Genotype 0: 28586 (92.55%)
Genotype 1: 1324 (4.29%)
Genotype 2: 881 (2.85%)
Genotype 9: 97 (0.31%)

Writing genotypes to ../data/1000g/populations/YRI.geno...


100%|██████████████████████████████████| 21664/21664 [00:00<00:00, 41390.28it/s]


Conversion complete for YRI!
Files created:
1. ../data/1000g/populations/YRI.snp - SNP information (21664 variants)
2. ../data/1000g/populations/YRI.ind - Sample information (108 samples)
3. ../data/1000g/populations/YRI.geno - Genotype data

Final genotype distribution:
Genotype 0: 2186118 (93.44%)
Genotype 1: 99371 (4.25%)
Genotype 2: 52447 (2.24%)
Genotype 9: 1776 (0.08%)


In [7]:
import os
import subprocess
from tqdm import tqdm
import pandas as pd

class MAFFilter:
    def __init__(self, data_dir='../data/1000g', output_dir=None):
        self.data_dir = data_dir
        self.output_dir = output_dir or os.path.join(data_dir, 'maf_filtered')
        os.makedirs(self.output_dir, exist_ok=True)
    
    def calculate_maf(self, vcf_file: str, output_file: str):
        """Calculate MAF for each variant and filter those with MAF < 5%"""
        print(f"Processing {vcf_file}")
        
        # Calculate allele frequencies using bcftools
        cmd = (f"bcftools +fill-tags {vcf_file} -- -t AF,MAF | "
               f"bcftools view -i 'MAF >= 0.05' -Oz -o {output_file}")
        
        print("Filtering variants with MAF < 5%...")
        subprocess.run(cmd, shell=True, check=True)
        
        # Index the filtered file
        subprocess.run(f"bcftools index {output_file}", shell=True, check=True)
        
        # Get statistics
        stats_cmd = f"bcftools stats {output_file}"
        stats = subprocess.run(stats_cmd, shell=True, capture_output=True, text=True).stdout
        
        return output_file, stats
    
    def filter_all_chromosomes(self):
        """Filter all chromosomes"""
        results = []
        
        for chrom in range(1, 23):
            input_vcf = os.path.join(self.data_dir, 
                                   f'filtered.chr{chrom}.phase3.vcf.gz')
            output_vcf = os.path.join(self.output_dir, 
                                    f'maf_filtered.chr{chrom}.phase3.vcf.gz')
            
            if not os.path.exists(input_vcf):
                print(f"Warning: Input file not found: {input_vcf}")
                continue
            
            # Calculate MAF and filter
            filtered_file, stats = self.calculate_maf(input_vcf, output_vcf)
            
            # Extract key statistics
            variant_counts = {}
            for line in stats.split('\n'):
                if line.startswith('SN'):
                    if 'number of records:' in line:
                        variant_counts['total'] = int(line.split()[-1])
            
            results.append({
                'chromosome': chrom,
                'filtered_variants': variant_counts.get('total', 0),
                'output_file': filtered_file
            })
        
        # Create summary
        summary_df = pd.DataFrame(results)
        summary_file = os.path.join(self.output_dir, 'maf_filtering_summary.csv')
        summary_df.to_csv(summary_file, index=False)
        
        print("\nMAF Filtering Summary:")
        print(summary_df)
        print(f"\nTotal variants after filtering: {summary_df['filtered_variants'].sum():,}")
        print(f"\nDetailed summary saved to: {summary_file}")
        
        return summary_df

def main():
    # Initialize filter
    maf_filter = MAFFilter()
    
    # Run filtering
    print("Starting MAF filtering for all chromosomes...")
    summary = maf_filter.filter_all_chromosomes()
    
    print("\nFiltering complete!")
    print("Filtered VCF files are saved in:", maf_filter.output_dir)

if __name__ == "__main__":
    main()

Starting MAF filtering for all chromosomes...
Processing ../data/1000g/filtered.chr1.phase3.vcf.gz
Filtering variants with MAF < 5%...
Processing ../data/1000g/filtered.chr2.phase3.vcf.gz
Filtering variants with MAF < 5%...
Processing ../data/1000g/filtered.chr3.phase3.vcf.gz
Filtering variants with MAF < 5%...
Processing ../data/1000g/filtered.chr4.phase3.vcf.gz
Filtering variants with MAF < 5%...
Processing ../data/1000g/filtered.chr5.phase3.vcf.gz
Filtering variants with MAF < 5%...
Processing ../data/1000g/filtered.chr6.phase3.vcf.gz
Filtering variants with MAF < 5%...
Processing ../data/1000g/filtered.chr7.phase3.vcf.gz
Filtering variants with MAF < 5%...
Processing ../data/1000g/filtered.chr8.phase3.vcf.gz
Filtering variants with MAF < 5%...
Processing ../data/1000g/filtered.chr9.phase3.vcf.gz
Filtering variants with MAF < 5%...
Processing ../data/1000g/filtered.chr10.phase3.vcf.gz
Filtering variants with MAF < 5%...
Processing ../data/1000g/filtered.chr11.phase3.vcf.gz
Filtering

In [11]:

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import Dict, Set, Tuple

class HapMapFilter:
    def __init__(self, hapmap_dir: str, common_snps_dir: str, output_dir: str):
        self.hapmap_dir = hapmap_dir
        self.common_snps_dir = common_snps_dir
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        
    def _load_common_snps(self) -> Dict[int, pd.DataFrame]:
        """Load common SNP information by chromosome"""
        common_snps = {}
        
        print("Loading common SNP positions...")
        for chrom in range(1, 23):
            # Read both position and alleles from common SNPs VCF
            vcf_file = os.path.join(self.common_snps_dir, f'common_snps.chr{chrom}.vcf.gz')
            cmd = f"bcftools query -f '%CHROM\t%POS\t%REF\t%ALT\n' {vcf_file}"
            import subprocess
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
            
            # Parse the output into a DataFrame
            snp_info = []
            for line in result.stdout.strip().split('\n'):
                if line:
                    chrom, pos, ref, alt = line.split('\t')
                    snp_info.append({
                        'chromosome': int(chrom),
                        'position': int(pos),
                        'ref': ref,
                        'alt': alt
                    })
            
            if snp_info:
                common_snps[chrom] = pd.DataFrame(snp_info)
                print(f"Chromosome {chrom}: {len(snp_info)} common SNPs")
            else:
                print(f"Warning: No SNPs found for chromosome {chrom}")
        
        return common_snps
    
    def _read_hapmap_data(self, population: str) -> Tuple[pd.DataFrame, pd.DataFrame, np.ndarray]:
        """Read HapMap3 data files for a population"""
        base_path = os.path.join(self.hapmap_dir, population)
        
        print(f"Reading data from {base_path}")
        
        # Read SNP information
        snp_data = pd.read_csv(f"{base_path}.snp", sep='\s+', header=None,
                              names=['SNP', 'chromosome', 'morgans', 'position', 'ref', 'alt'])
        print(f"Read {len(snp_data)} SNPs")
        print("Sample SNP data:")
        print(snp_data.head())
        
        # Read sample information
        ind_data = pd.read_csv(f"{base_path}.ind", sep='\s+', header=None,
                              names=['sample', 'sex', 'population'])
        print(f"Read {len(ind_data)} samples")
        
        # Read genotype data
        print("Reading genotype data...")
        geno_data = []
        with open(f"{base_path}.geno", 'r') as f:
            for line in tqdm(f):
                geno_data.append([int(g) for g in line.strip()])
        geno_array = np.array(geno_data)
        print(f"Genotype array shape: {geno_array.shape}")
        
        return snp_data, ind_data, geno_array
    
    def filter_population(self, population: str):
        """Filter HapMap3 data for a population to keep only common SNPs"""
        print(f"\nProcessing population: {population}")
        
        # Load common SNP positions
        common_snps = self._load_common_snps()
        
        # Read HapMap3 data
        print("\nReading HapMap3 data...")
        snp_data, ind_data, geno_array = self._read_hapmap_data(population)
        
        # Debug: Print chromosome distribution in HapMap data
        print("\nChromosome distribution in HapMap data:")
        print(snp_data['chromosome'].value_counts().sort_index())
        
        # Create mask for common SNPs
        print("\nFinding common SNPs...")
        keep_mask = np.zeros(len(snp_data), dtype=bool)
        
        for chrom in range(1, 23):
            # Get HapMap SNPs for this chromosome
            chr_mask = snp_data['chromosome'] == chrom
            chr_snps = snp_data[chr_mask]
            
            if chrom in common_snps:
                common_chr_snps = common_snps[chrom]
                
                # Debug information
                print(f"\nChromosome {chrom}:")
                print(f"HapMap SNPs: {len(chr_snps)}")
                print(f"Common SNPs: {len(common_chr_snps)}")
                
                # Match positions
                matches = chr_snps['position'].isin(common_chr_snps['position'])
                keep_mask[chr_mask] = matches
                
                print(f"Matching positions: {sum(matches)}")
                
                # Debug: Show some matching examples
                if sum(matches) > 0:
                    print("\nSample matching SNPs:")
                    matched_hapmap = chr_snps[matches].head()
                    matched_common = common_chr_snps[common_chr_snps['position'].isin(matched_hapmap['position'])].head()
                    print("HapMap SNPs:")
                    print(matched_hapmap[['position', 'ref', 'alt']])
                    print("Common SNPs:")
                    print(matched_common[['position', 'ref', 'alt']])
        
        # Apply filtering
        filtered_snps = snp_data[keep_mask]
        filtered_geno = geno_array[keep_mask]
        
        if len(filtered_snps) == 0:
            raise ValueError("No matching SNPs found! Please check the chromosome numbering and position matching logic.")
        
        # Save filtered data
        output_prefix = os.path.join(self.output_dir, f"{population}_filtered")
        
        filtered_snps.to_csv(f"{output_prefix}.snp", sep='\t', index=False, header=False)
        ind_data.to_csv(f"{output_prefix}.ind", sep='\t', index=False, header=False)
        
        print("\nSaving filtered genotypes...")
        with open(f"{output_prefix}.geno", 'w') as f:
            for row in tqdm(filtered_geno):
                f.write(''.join(map(str, row)) + '\n')
        
        print(f"\nFiltering statistics for {population}:")
        print(f"Original SNPs: {len(snp_data)}")
        print(f"Filtered SNPs: {len(filtered_snps)}")
        print(f"Retention rate: {len(filtered_snps)/len(snp_data)*100:.2f}%")
        
        return filtered_snps, ind_data, filtered_geno

    def filter_all_populations(self, populations=None):
        if populations is None:
            populations = ['ASW', 'CEU', 'CHB', 'JPT', 'LWK', 'MXL', 'TSI', 'YRI']
        
        for population in populations:
            try:
                self.filter_population(population)
            except Exception as e:
                print(f"Error processing population {population}: {e}")
                import traceback
                traceback.print_exc()

if __name__ == "__main__":
    # Configure paths
    HAPMAP_DIR = '/Users/jingl1/Desktop/CMU/02704/hw_data/02704_data'  # HapMap3种群数据目录
    COMMON_SNPS_DIR = "../data/common_snps"      # 共同SNP目录
    OUTPUT_DIR = "../data/hapmap3_filtered"      # 输出目录
    
    # Initialize and run filter
    hapmap_filter = HapMapFilter(HAPMAP_DIR, COMMON_SNPS_DIR, OUTPUT_DIR)
    hapmap_filter.filter_all_populations()


Processing population: ASW
Loading common SNP positions...
Chromosome 1: 1617 common SNPs
Chromosome 2: 1786 common SNPs
Chromosome 3: 1446 common SNPs
Chromosome 4: 1366 common SNPs
Chromosome 5: 1325 common SNPs
Chromosome 6: 1423 common SNPs
Chromosome 7: 1215 common SNPs
Chromosome 8: 1382 common SNPs
Chromosome 9: 1038 common SNPs
Chromosome 10: 1128 common SNPs
Chromosome 11: 1154 common SNPs
Chromosome 12: 1046 common SNPs
Chromosome 13: 773 common SNPs
Chromosome 14: 667 common SNPs
Chromosome 15: 637 common SNPs
Chromosome 16: 835 common SNPs
Chromosome 17: 664 common SNPs
Chromosome 18: 598 common SNPs
Chromosome 19: 367 common SNPs
Chromosome 20: 571 common SNPs
Chromosome 21: 340 common SNPs
Chromosome 22: 286 common SNPs

Reading HapMap3 data...
Reading data from /Users/jingl1/Desktop/CMU/02704/hw_data/02704_data/ASW
Read 718848 SNPs
Sample SNP data:
          SNP  chromosome  morgans  position ref alt
0   rs3131972           1      0.0    742584   G   A
1   rs3131969    

718848it [00:02, 264627.22it/s]
Traceback (most recent call last):
  File "/var/folders/96/2s64979965l0x1hmvz6rctxh0000gp/T/ipykernel_35761/819443221.py", line 154, in filter_all_populations
    self.filter_population(population)
  File "/var/folders/96/2s64979965l0x1hmvz6rctxh0000gp/T/ipykernel_35761/819443221.py", line 128, in filter_population
    raise ValueError("No matching SNPs found! Please check the chromosome numbering and position matching logic.")
ValueError: No matching SNPs found! Please check the chromosome numbering and position matching logic.


Genotype array shape: (718848, 49)

Chromosome distribution in HapMap data:
chromosome
1     55983
2     58409
3     48655
4     43809
5     45045
6     46016
7     38578
8     38291
9     32252
10    36847
11    36514
12    33528
13    26595
14    22956
15    21459
16    22191
17    19210
18    21196
19    13472
20    18359
21    10281
22     9447
23    19755
Name: count, dtype: int64

Finding common SNPs...
Error processing population ASW: No matching SNPs found! Please check the chromosome numbering and position matching logic.

Processing population: CEU
Loading common SNP positions...
Chromosome 1: 1617 common SNPs
Chromosome 2: 1786 common SNPs
Chromosome 3: 1446 common SNPs
Chromosome 4: 1366 common SNPs
Chromosome 5: 1325 common SNPs
Chromosome 6: 1423 common SNPs
Chromosome 7: 1215 common SNPs
Chromosome 8: 1382 common SNPs
Chromosome 9: 1038 common SNPs
Chromosome 10: 1128 common SNPs
Chromosome 11: 1154 common SNPs
Chromosome 12: 1046 common SNPs
Chromosome 13: 773 common SN

718848it [00:05, 124886.73it/s]
Traceback (most recent call last):
  File "/var/folders/96/2s64979965l0x1hmvz6rctxh0000gp/T/ipykernel_35761/819443221.py", line 154, in filter_all_populations
    self.filter_population(population)
  File "/var/folders/96/2s64979965l0x1hmvz6rctxh0000gp/T/ipykernel_35761/819443221.py", line 128, in filter_population
    raise ValueError("No matching SNPs found! Please check the chromosome numbering and position matching logic.")
ValueError: No matching SNPs found! Please check the chromosome numbering and position matching logic.


Genotype array shape: (718848, 112)

Chromosome distribution in HapMap data:
chromosome
1     55983
2     58409
3     48655
4     43809
5     45045
6     46016
7     38578
8     38291
9     32252
10    36847
11    36514
12    33528
13    26595
14    22956
15    21459
16    22191
17    19210
18    21196
19    13472
20    18359
21    10281
22     9447
23    19755
Name: count, dtype: int64

Finding common SNPs...
Error processing population CEU: No matching SNPs found! Please check the chromosome numbering and position matching logic.

Processing population: CHB
Loading common SNP positions...
Chromosome 1: 1617 common SNPs
Chromosome 2: 1786 common SNPs
Chromosome 3: 1446 common SNPs
Chromosome 4: 1366 common SNPs
Chromosome 5: 1325 common SNPs
Chromosome 6: 1423 common SNPs
Chromosome 7: 1215 common SNPs
Chromosome 8: 1382 common SNPs
Chromosome 9: 1038 common SNPs
Chromosome 10: 1128 common SNPs
Chromosome 11: 1154 common SNPs
Chromosome 12: 1046 common SNPs
Chromosome 13: 773 common S

...

718848it [00:04, 157288.55it/s]
Traceback (most recent call last):
  File "/var/folders/96/2s64979965l0x1hmvz6rctxh0000gp/T/ipykernel_35761/819443221.py", line 154, in filter_all_populations
    self.filter_population(population)
  File "/var/folders/96/2s64979965l0x1hmvz6rctxh0000gp/T/ipykernel_35761/819443221.py", line 128, in filter_population
    raise ValueError("No matching SNPs found! Please check the chromosome numbering and position matching logic.")
ValueError: No matching SNPs found! Please check the chromosome numbering and position matching logic.


Genotype array shape: (718848, 88)

Chromosome distribution in HapMap data:
chromosome
1     55983
2     58409
3     48655
4     43809
5     45045
6     46016
7     38578
8     38291
9     32252
10    36847
11    36514
12    33528
13    26595
14    22956
15    21459
16    22191
17    19210
18    21196
19    13472
20    18359
21    10281
22     9447
23    19755
Name: count, dtype: int64

Finding common SNPs...
Error processing population TSI: No matching SNPs found! Please check the chromosome numbering and position matching logic.

Processing population: YRI
Loading common SNP positions...
Chromosome 1: 1617 common SNPs
Chromosome 2: 1786 common SNPs
Chromosome 3: 1446 common SNPs
Chromosome 4: 1366 common SNPs
Chromosome 5: 1325 common SNPs
Chromosome 6: 1423 common SNPs
Chromosome 7: 1215 common SNPs
Chromosome 8: 1382 common SNPs
Chromosome 9: 1038 common SNPs
Chromosome 10: 1128 common SNPs
Chromosome 11: 1154 common SNPs
Chromosome 12: 1046 common SNPs
Chromosome 13: 773 common SN

718848it [00:05, 124894.56it/s]


Genotype array shape: (718848, 113)

Chromosome distribution in HapMap data:
chromosome
1     55983
2     58409
3     48655
4     43809
5     45045
6     46016
7     38578
8     38291
9     32252
10    36847
11    36514
12    33528
13    26595
14    22956
15    21459
16    22191
17    19210
18    21196
19    13472
20    18359
21    10281
22     9447
23    19755
Name: count, dtype: int64

Finding common SNPs...
Error processing population YRI: No matching SNPs found! Please check the chromosome numbering and position matching logic.


Traceback (most recent call last):
  File "/var/folders/96/2s64979965l0x1hmvz6rctxh0000gp/T/ipykernel_35761/819443221.py", line 154, in filter_all_populations
    self.filter_population(population)
  File "/var/folders/96/2s64979965l0x1hmvz6rctxh0000gp/T/ipykernel_35761/819443221.py", line 128, in filter_population
    raise ValueError("No matching SNPs found! Please check the chromosome numbering and position matching logic.")
ValueError: No matching SNPs found! Please check the chromosome numbering and position matching logic.


In [47]:
from CB_02704 import *
import pandas as pd
import numpy as np
import os
import logging
from typing import Dict, Tuple, Set

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class DatasetComparator:
    def __init__(self):
        self.populations = ['ASW', 'CEU', 'CHB', 'CHD', 'GIH', 'JPT', 'LWK', 'MKK', 'MXL', 'TSI', 'YRI']
        
    def compare_datasets(self, pop: str) -> Dict:
        """比较HapMap和1000G数据集的SNP重叠情况"""
        results = {
            'population': pop,
            'hapmap_snps': 0,
            'common_snps': 0,
            'chromosome_stats': {}
        }
        
        # 读取HapMap数据
        try:
            hapmap_df = read_snp_pop(pop)
            results['hapmap_snps'] = len(hapmap_df)
            logger.info(f"Loaded {len(hapmap_df)} SNPs from HapMap for {pop}")
            
            # 获取所有SNP ID
            hapmap_snps = set(hapmap_df.index)
            
            # 使用SNPs全局变量（从课程文档中看到这个是HapMap3的所有SNP信息）
            all_snps = set(SNPs.index)
            
            # 计算重叠
            common_snps = hapmap_snps.intersection(all_snps)
            results['common_snps'] = len(common_snps)
            results['overlap_percentage'] = (len(common_snps) / len(hapmap_snps) * 100) \
                                          if len(hapmap_snps) > 0 else 0
            
            # 按染色体统计
            for chrom in range(1, 23):
                hapmap_chr = hapmap_df[hapmap_df['chromosome'] == chrom]
                all_chr = SNPs[SNPs['chromosome'] == chrom]
                
                common_chr = set(hapmap_chr.index).intersection(set(all_chr.index))
                
                results['chromosome_stats'][chrom] = {
                    'hapmap_snps': len(hapmap_chr),
                    'total_snps': len(all_chr),
                    'common_snps': len(common_chr),
                    'overlap_percentage': (len(common_chr) / len(hapmap_chr) * 100) \
                                        if len(hapmap_chr) > 0 else 0
                }
                
                logger.info(f"Chromosome {chrom}: Found {len(common_chr)} common SNPs")
                
        except Exception as e:
            logger.error(f"Error processing population {pop}: {e}")
            raise
            
        return results

    def analyze_population(self, pop: str) -> Tuple[pd.Series, pd.DataFrame]:
        """分析单个人群的数据集重叠情况，并返回详细的染色体级别统计信息"""
        results = self.compare_datasets(pop)
        
        # 创建染色体级别的DataFrame
        chr_stats = pd.DataFrame.from_dict(results['chromosome_stats'], orient='index')
        chr_stats.index.name = 'Chromosome'
        
        # 添加总体统计信息
        summary = pd.Series({
            'Population': pop,
            'Total HapMap SNPs': results['hapmap_snps'],
            'Total Common SNPs': results['common_snps'],
            'Overall Overlap %': results['overlap_percentage']
        })
        
        return summary, chr_stats

def main():
    """主函数"""
    comparator = DatasetComparator()
    
    # 分析所有人群
    all_summaries = []
    all_chr_stats = {}
    
    for pop in comparator.populations:
        logger.info(f"Analyzing population: {pop}")
        try:
            summary, chr_stats = comparator.analyze_population(pop)
            all_summaries.append(summary)
            all_chr_stats[pop] = chr_stats
            
            print(f"\nResults for {pop}:")
            print("=" * 80)
            print(summary.to_string())
            print("\nChromosome-level Statistics:")
            print(chr_stats.to_string())
            
        except Exception as e:
            logger.error(f"Failed to analyze {pop}: {e}")
            continue
    
    # 合并所有结果
    overall_summary = pd.DataFrame(all_summaries)
    
    # 保存结果
    overall_summary.to_csv('population_comparison_summary.csv', index=False)
    
    for pop, stats in all_chr_stats.items():
        stats.to_csv(f'{pop}_chromosome_stats.csv')
    
    logger.info("Analysis complete. Results saved to CSV files.")
    
if __name__ == "__main__":
    main()

INFO:__main__:Analyzing population: ASW
INFO:__main__:Loaded 718848 SNPs from HapMap for ASW
INFO:__main__:Chromosome 1: Found 55983 common SNPs
INFO:__main__:Chromosome 2: Found 58409 common SNPs
INFO:__main__:Chromosome 3: Found 48655 common SNPs
INFO:__main__:Chromosome 4: Found 43809 common SNPs
INFO:__main__:Chromosome 5: Found 45045 common SNPs
INFO:__main__:Chromosome 6: Found 46016 common SNPs
INFO:__main__:Chromosome 7: Found 38578 common SNPs
INFO:__main__:Chromosome 8: Found 38291 common SNPs
INFO:__main__:Chromosome 9: Found 32252 common SNPs
INFO:__main__:Chromosome 10: Found 36847 common SNPs
INFO:__main__:Chromosome 11: Found 36514 common SNPs
INFO:__main__:Chromosome 12: Found 33528 common SNPs
INFO:__main__:Chromosome 13: Found 26595 common SNPs
INFO:__main__:Chromosome 14: Found 22956 common SNPs
INFO:__main__:Chromosome 15: Found 21459 common SNPs
INFO:__main__:Chromosome 16: Found 22191 common SNPs
INFO:__main__:Chromosome 17: Found 19210 common SNPs
INFO:__main__:C


Results for ASW:
Population              ASW
Total HapMap SNPs    718848
Total Common SNPs    718848
Overall Overlap %     100.0

Chromosome-level Statistics:
            hapmap_snps  total_snps  common_snps  overlap_percentage
Chromosome                                                          
1                 55983       55983        55983               100.0
2                 58409       58409        58409               100.0
3                 48655       48655        48655               100.0
4                 43809       43809        43809               100.0
5                 45045       45045        45045               100.0
6                 46016       46016        46016               100.0
7                 38578       38578        38578               100.0
8                 38291       38291        38291               100.0
9                 32252       32252        32252               100.0
10                36847       36847        36847               100.0
11          

INFO:__main__:Loaded 718848 SNPs from HapMap for CEU
INFO:__main__:Chromosome 1: Found 55983 common SNPs
INFO:__main__:Chromosome 2: Found 58409 common SNPs
INFO:__main__:Chromosome 3: Found 48655 common SNPs
INFO:__main__:Chromosome 4: Found 43809 common SNPs
INFO:__main__:Chromosome 5: Found 45045 common SNPs
INFO:__main__:Chromosome 6: Found 46016 common SNPs
INFO:__main__:Chromosome 7: Found 38578 common SNPs
INFO:__main__:Chromosome 8: Found 38291 common SNPs
INFO:__main__:Chromosome 9: Found 32252 common SNPs
INFO:__main__:Chromosome 10: Found 36847 common SNPs
INFO:__main__:Chromosome 11: Found 36514 common SNPs
INFO:__main__:Chromosome 12: Found 33528 common SNPs
INFO:__main__:Chromosome 13: Found 26595 common SNPs
INFO:__main__:Chromosome 14: Found 22956 common SNPs
INFO:__main__:Chromosome 15: Found 21459 common SNPs
INFO:__main__:Chromosome 16: Found 22191 common SNPs
INFO:__main__:Chromosome 17: Found 19210 common SNPs
INFO:__main__:Chromosome 18: Found 21196 common SNPs
IN


Results for CEU:
Population              CEU
Total HapMap SNPs    718848
Total Common SNPs    718848
Overall Overlap %     100.0

Chromosome-level Statistics:
            hapmap_snps  total_snps  common_snps  overlap_percentage
Chromosome                                                          
1                 55983       55983        55983               100.0
2                 58409       58409        58409               100.0
3                 48655       48655        48655               100.0
4                 43809       43809        43809               100.0
5                 45045       45045        45045               100.0
6                 46016       46016        46016               100.0
7                 38578       38578        38578               100.0
8                 38291       38291        38291               100.0
9                 32252       32252        32252               100.0
10                36847       36847        36847               100.0
11          

INFO:__main__:Loaded 718848 SNPs from HapMap for CHB
INFO:__main__:Chromosome 1: Found 55983 common SNPs
INFO:__main__:Chromosome 2: Found 58409 common SNPs
INFO:__main__:Chromosome 3: Found 48655 common SNPs
INFO:__main__:Chromosome 4: Found 43809 common SNPs
INFO:__main__:Chromosome 5: Found 45045 common SNPs
INFO:__main__:Chromosome 6: Found 46016 common SNPs
INFO:__main__:Chromosome 7: Found 38578 common SNPs
INFO:__main__:Chromosome 8: Found 38291 common SNPs
INFO:__main__:Chromosome 9: Found 32252 common SNPs
INFO:__main__:Chromosome 10: Found 36847 common SNPs
INFO:__main__:Chromosome 11: Found 36514 common SNPs
INFO:__main__:Chromosome 12: Found 33528 common SNPs
INFO:__main__:Chromosome 13: Found 26595 common SNPs
INFO:__main__:Chromosome 14: Found 22956 common SNPs
INFO:__main__:Chromosome 15: Found 21459 common SNPs
INFO:__main__:Chromosome 16: Found 22191 common SNPs
INFO:__main__:Chromosome 17: Found 19210 common SNPs
INFO:__main__:Chromosome 18: Found 21196 common SNPs
IN

...


Results for MXL:
Population              MXL
Total HapMap SNPs    718848
Total Common SNPs    718848
Overall Overlap %     100.0

Chromosome-level Statistics:
            hapmap_snps  total_snps  common_snps  overlap_percentage
Chromosome                                                          
1                 55983       55983        55983               100.0
2                 58409       58409        58409               100.0
3                 48655       48655        48655               100.0
4                 43809       43809        43809               100.0
5                 45045       45045        45045               100.0
6                 46016       46016        46016               100.0
7                 38578       38578        38578               100.0
8                 38291       38291        38291               100.0
9                 32252       32252        32252               100.0
10                36847       36847        36847               100.0
11          

INFO:__main__:Loaded 718848 SNPs from HapMap for TSI
INFO:__main__:Chromosome 1: Found 55983 common SNPs
INFO:__main__:Chromosome 2: Found 58409 common SNPs
INFO:__main__:Chromosome 3: Found 48655 common SNPs
INFO:__main__:Chromosome 4: Found 43809 common SNPs
INFO:__main__:Chromosome 5: Found 45045 common SNPs
INFO:__main__:Chromosome 6: Found 46016 common SNPs
INFO:__main__:Chromosome 7: Found 38578 common SNPs
INFO:__main__:Chromosome 8: Found 38291 common SNPs
INFO:__main__:Chromosome 9: Found 32252 common SNPs
INFO:__main__:Chromosome 10: Found 36847 common SNPs
INFO:__main__:Chromosome 11: Found 36514 common SNPs
INFO:__main__:Chromosome 12: Found 33528 common SNPs
INFO:__main__:Chromosome 13: Found 26595 common SNPs
INFO:__main__:Chromosome 14: Found 22956 common SNPs
INFO:__main__:Chromosome 15: Found 21459 common SNPs
INFO:__main__:Chromosome 16: Found 22191 common SNPs
INFO:__main__:Chromosome 17: Found 19210 common SNPs
INFO:__main__:Chromosome 18: Found 21196 common SNPs
IN


Results for TSI:
Population              TSI
Total HapMap SNPs    718848
Total Common SNPs    718848
Overall Overlap %     100.0

Chromosome-level Statistics:
            hapmap_snps  total_snps  common_snps  overlap_percentage
Chromosome                                                          
1                 55983       55983        55983               100.0
2                 58409       58409        58409               100.0
3                 48655       48655        48655               100.0
4                 43809       43809        43809               100.0
5                 45045       45045        45045               100.0
6                 46016       46016        46016               100.0
7                 38578       38578        38578               100.0
8                 38291       38291        38291               100.0
9                 32252       32252        32252               100.0
10                36847       36847        36847               100.0
11          

INFO:__main__:Loaded 718848 SNPs from HapMap for YRI
INFO:__main__:Chromosome 1: Found 55983 common SNPs
INFO:__main__:Chromosome 2: Found 58409 common SNPs
INFO:__main__:Chromosome 3: Found 48655 common SNPs
INFO:__main__:Chromosome 4: Found 43809 common SNPs
INFO:__main__:Chromosome 5: Found 45045 common SNPs
INFO:__main__:Chromosome 6: Found 46016 common SNPs
INFO:__main__:Chromosome 7: Found 38578 common SNPs
INFO:__main__:Chromosome 8: Found 38291 common SNPs
INFO:__main__:Chromosome 9: Found 32252 common SNPs
INFO:__main__:Chromosome 10: Found 36847 common SNPs
INFO:__main__:Chromosome 11: Found 36514 common SNPs
INFO:__main__:Chromosome 12: Found 33528 common SNPs
INFO:__main__:Chromosome 13: Found 26595 common SNPs
INFO:__main__:Chromosome 14: Found 22956 common SNPs
INFO:__main__:Chromosome 15: Found 21459 common SNPs
INFO:__main__:Chromosome 16: Found 22191 common SNPs
INFO:__main__:Chromosome 17: Found 19210 common SNPs
INFO:__main__:Chromosome 18: Found 21196 common SNPs
IN


Results for YRI:
Population              YRI
Total HapMap SNPs    718848
Total Common SNPs    718848
Overall Overlap %     100.0

Chromosome-level Statistics:
            hapmap_snps  total_snps  common_snps  overlap_percentage
Chromosome                                                          
1                 55983       55983        55983               100.0
2                 58409       58409        58409               100.0
3                 48655       48655        48655               100.0
4                 43809       43809        43809               100.0
5                 45045       45045        45045               100.0
6                 46016       46016        46016               100.0
7                 38578       38578        38578               100.0
8                 38291       38291        38291               100.0
9                 32252       32252        32252               100.0
10                36847       36847        36847               100.0
11          

In [52]:
from CB_02704 import *
import pandas as pd
import numpy as np
import subprocess
import logging
import os
from pathlib import Path

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def read_1000g_snps(chrom: int, base_path: str) -> pd.DataFrame:
    """读取1000G VCF文件并返回SNP信息，包括位置信息"""
    vcf_path = Path(base_path) / f"maf_filtered.chr{chrom}.phase3.vcf.gz"
    
    if not vcf_path.exists():
        logger.warning(f"File not found: {vcf_path}")
        return pd.DataFrame()
        
    try:
        # 使用bcftools提取SNP信息，包括位置信息
        cmd = f"bcftools view -H {vcf_path} | cut -f1-5"
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        
        if result.returncode != 0:
            logger.error(f"bcftools error: {result.stderr}")
            return pd.DataFrame()
        
        # 将输出转换为DataFrame
        rows = []
        for line in result.stdout.strip().split('\n'):
            if line:
                chrom, pos, snp_id, ref, alt = line.split('\t')
                rows.append({
                    'chromosome': int(chrom.replace('chr', '')),
                    'position': int(pos),
                    'snp_id': snp_id if snp_id != '.' else f"chr{chrom}:{pos}",
                    'ref': ref,
                    'alt': alt
                })
        
        df = pd.DataFrame(rows)
        return df
    
    except Exception as e:
        logger.error(f"Error reading chromosome {chrom} from 1000G: {e}")
        return pd.DataFrame()

def compare_snp_sets(kg_base_path: str="../data/1000g/maf_filtered"):
    """比较HapMap和1000G的SNP集合，使用ID和位置信息"""
    results = []
    total_stats = {'hapmap': 0, '1000g': 0, 'matched': 0}
    
    # 处理每条染色体
    for chrom in range(1, 23):
        logger.info(f"\nProcessing chromosome {chrom}")
        
        # 获取HapMap数据
        hapmap_chr = SNPs[SNPs['chromosome'] == chrom].copy()
        hapmap_chr['pos_key'] = hapmap_chr.apply(lambda x: f"chr{x['chromosome']}:{x['position']}", axis=1)
        
        # 获取1000G数据
        kg_chr = read_1000g_snps(chrom, kg_base_path)
        if kg_chr.empty:
            continue
            
        # 创建位置键
        kg_chr['pos_key'] = kg_chr.apply(lambda x: f"chr{x['chromosome']}:{x['position']}", axis=1)
        
        # 1. 首先通过rs ID匹配
        hapmap_rs_ids = set(hapmap_chr.index)
        kg_rs_ids = set(kg_chr[kg_chr['snp_id'].str.startswith('rs')]['snp_id'])
        id_matches = hapmap_rs_ids.intersection(kg_rs_ids)
        
        # 2. 然后通过位置匹配
        hapmap_positions = set(hapmap_chr['pos_key'])
        kg_positions = set(kg_chr['pos_key'])
        pos_matches = hapmap_positions.intersection(kg_positions)
        
        # 统计
        n_hapmap = len(hapmap_chr)
        n_1000g = len(kg_chr)
        n_id_matches = len(id_matches)
        n_pos_matches = len(pos_matches)
        n_total_matches = len(set(id_matches).union(pos_matches))
        
        results.append({
            'chromosome': chrom,
            'hapmap_snps': n_hapmap,
            '1000g_snps': n_1000g,
            'rs_matches': n_id_matches,
            'pos_matches': n_pos_matches,
            'total_matches': n_total_matches,
            'hapmap_overlap': (n_total_matches / n_hapmap * 100) if n_hapmap > 0 else 0,
            '1000g_overlap': (n_total_matches / n_1000g * 100) if n_1000g > 0 else 0
        })
        
        # 更新总计
        total_stats['hapmap'] += n_hapmap
        total_stats['1000g'] += n_1000g
        total_stats['matched'] += n_total_matches
        
        # 输出染色体级别的统计
        logger.info(f"Chromosome {chrom} summary:")
        logger.info(f"  HapMap SNPs: {n_hapmap:,}")
        logger.info(f"  1000G SNPs: {n_1000g:,}")
        logger.info(f"  Matched by rs ID: {n_id_matches:,}")
        logger.info(f"  Matched by position: {n_pos_matches:,}")
        logger.info(f"  Total matches: {n_total_matches:,}")
    
    # 创建汇总DataFrame
    summary_df = pd.DataFrame(results)
    summary_df.set_index('chromosome', inplace=True)
    
    # 添加总计行
    if not summary_df.empty:
        summary_df.loc['Total'] = {
            'hapmap_snps': total_stats['hapmap'],
            '1000g_snps': total_stats['1000g'],
            'rs_matches': sum(r['rs_matches'] for r in results),
            'pos_matches': sum(r['pos_matches'] for r in results),
            'total_matches': total_stats['matched'],
            'hapmap_overlap': (total_stats['matched'] / total_stats['hapmap'] * 100),
            '1000g_overlap': (total_stats['matched'] / total_stats['1000g'] * 100)
        }
    
    return summary_df

def main():
    print("\nStarting SNP dataset comparison...")
    summary_df = compare_snp_sets()
    
    print("\nComparison Summary:")
    print("=" * 80)
    print(summary_df)
    
    # 保存结果
    summary_df.to_csv('snp_comparison_summary.csv')
    print("\nResults saved to snp_comparison_summary.csv")

if __name__ == "__main__":
    main()

INFO:__main__:
Processing chromosome 1



Starting SNP dataset comparison...


INFO:__main__:Chromosome 1 summary:
INFO:__main__:  HapMap SNPs: 55,983
INFO:__main__:  1000G SNPs: 633,841
INFO:__main__:  Matched by rs ID: 0
INFO:__main__:  Matched by position: 172
INFO:__main__:  Total matches: 172
INFO:__main__:
Processing chromosome 2
INFO:__main__:Chromosome 2 summary:
INFO:__main__:  HapMap SNPs: 58,409
INFO:__main__:  1000G SNPs: 675,710
INFO:__main__:  Matched by rs ID: 0
INFO:__main__:  Matched by position: 182
INFO:__main__:  Total matches: 182
INFO:__main__:
Processing chromosome 3
INFO:__main__:Chromosome 3 summary:
INFO:__main__:  HapMap SNPs: 48,655
INFO:__main__:  1000G SNPs: 583,523
INFO:__main__:  Matched by rs ID: 0
INFO:__main__:  Matched by position: 139
INFO:__main__:  Total matches: 139
INFO:__main__:
Processing chromosome 4
INFO:__main__:Chromosome 4 summary:
INFO:__main__:  HapMap SNPs: 43,809
INFO:__main__:  1000G SNPs: 603,946
INFO:__main__:  Matched by rs ID: 0
INFO:__main__:  Matched by position: 144
INFO:__main__:  Total matches: 144
INF


Comparison Summary:
            hapmap_snps  1000g_snps  rs_matches  pos_matches  total_matches  \
chromosome                                                                    
1                 55983      633841           0          172            172   
2                 58409      675710           0          182            182   
3                 48655      583523           0          139            139   
4                 43809      603946           0          144            144   
5                 45045      512157           0          150            150   
6                 46016      548585           0          177            177   
7                 38578      482234           0          132            132   
8                 38291      447449           0          126            126   
9                 32252      353854           0          113            113   
10                36847      416405           0          114            114   
11                36514      40

In [54]:
from CB_02704 import *
import pandas as pd
import numpy as np
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def debug_chromosome_data(chrom: int = 1):
    """调试指定染色体的数据格式"""
    # 获取该染色体的SNP范围
    start, stop = get_chr_range(chrom)
    logger.info(f"Chromosome {chrom} range: {start}-{stop}")
    
    # 获取HapMap数据
    hapmap_chr = SNPs.iloc[start:stop]
    logger.info(f"\nHapMap SNPs for chr{chrom}:")
    logger.info(f"Total SNPs: {len(hapmap_chr)}")
    logger.info("\nFirst 5 SNPs:")
    print(hapmap_chr.head())
    
    # 统计数据
    logger.info(f"\nHapMap chromosome stats:")
    logger.info(f"Number of unique positions: {len(hapmap_chr['position'].unique())}")
    logger.info(f"Number of unique SNP IDs: {len(hapmap_chr.index.unique())}")

def main():
    # 测试前几条染色体
    for chrom in range(1, 4):
        debug_chromosome_data(chrom)
        print("\n" + "="*80 + "\n")

if __name__ == "__main__":
    main()

INFO:__main__:Chromosome 1 range: 0-55983
INFO:__main__:
HapMap SNPs for chr1:
INFO:__main__:Total SNPs: 55983
INFO:__main__:
First 5 SNPs:
INFO:__main__:
HapMap chromosome stats:
INFO:__main__:Number of unique positions: 55861
INFO:__main__:Number of unique SNP IDs: 55983
INFO:__main__:Chromosome 2 range: 55983-114392
INFO:__main__:
HapMap SNPs for chr2:
INFO:__main__:Total SNPs: 58409
INFO:__main__:
First 5 SNPs:
INFO:__main__:
HapMap chromosome stats:
INFO:__main__:Number of unique positions: 58275
INFO:__main__:Number of unique SNP IDs: 58409
INFO:__main__:Chromosome 3 range: 114392-163047
INFO:__main__:
HapMap SNPs for chr3:
INFO:__main__:Total SNPs: 48655
INFO:__main__:
First 5 SNPs:
INFO:__main__:
HapMap chromosome stats:
INFO:__main__:Number of unique positions: 48546
INFO:__main__:Number of unique SNP IDs: 48655


            chromosome  morgans  position ref alt
rs3131972            1      0.0    742584   G   A
rs3131969            1      0.0    744045   G   A
rs3131967            1      0.0    744197   C   T
rs1048488            1      0.0    750775   T   C
rs12562034           1      0.0    758311   G   A


            chromosome  morgans  position ref alt
rs10165221           2      0.0     40814   C   T
rs7580475            2      0.0     90190   T   C
rs300803             2      0.0     91514   G   C
rs7588043            2      0.0     94979   C   T
rs436446             2      0.0     96409   C   T


            chromosome  morgans  position ref alt
rs13060385           3      0.0     36495   A   G
rs9713920            3      0.0     41083   T   C
rs9681213            3      0.0     41894   G   A
rs9682794            3      0.0     45895   T   C
rs12637617           3      0.0     46404   T   C




In [55]:
from CB_02704 import *
import pandas as pd
import numpy as np
import subprocess
import logging
from pathlib import Path

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def read_1000g_snps(chrom: int, base_path: str) -> pd.DataFrame:
    """读取1000G VCF文件并返回SNP信息，规范化格式以匹配HapMap"""
    vcf_path = Path(base_path) / f"maf_filtered.chr{chrom}.phase3.vcf.gz"
    
    if not vcf_path.exists():
        logger.warning(f"File not found: {vcf_path}")
        return pd.DataFrame()
        
    try:
        cmd = f"bcftools view -H {vcf_path} | cut -f1-5"
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        
        rows = []
        for line in result.stdout.strip().split('\n'):
            if line:
                chrom_str, pos, snp_id, ref, alt = line.split('\t')
                # 只保留标准rs ID的SNP
                if snp_id.startswith('rs'):
                    rows.append({
                        'chromosome': int(chrom_str),
                        'position': int(pos),
                        'snp_id': snp_id,
                        'ref': ref,
                        'alt': alt
                    })
        
        df = pd.DataFrame(rows)
        if not df.empty:
            df.set_index('snp_id', inplace=True)
            # 确保ref和alt是大写的
            df['ref'] = df['ref'].str.upper()
            df['alt'] = df['alt'].str.upper()
        return df
    
    except Exception as e:
        logger.error(f"Error reading chromosome {chrom} from 1000G: {e}")
        return pd.DataFrame()

def check_allele_match(row1, row2):
    """检查两个SNP的等位基因是否匹配（考虑互补和顺序）"""
    alleles1 = {row1['ref'], row1['alt']}
    alleles2 = {row2['ref'], row2['alt']}
    
    # 直接匹配
    if alleles1 == alleles2:
        return True
    
    # 检查互补
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    alleles2_complement = {complement.get(base, base) for base in alleles2}
    return alleles1 == alleles2_complement

def compare_snp_sets(kg_base_path: str="../data/1000g/maf_filtered"):
    """比较HapMap和1000G的SNP集合，确保正确的位置和等位基因匹配"""
    results = []
    
    for chrom in range(1, 23):
        logger.info(f"\nProcessing chromosome {chrom}")
        
        # 获取HapMap数据
        hapmap_chr = SNPs[SNPs['chromosome'] == chrom].copy()
        logger.info(f"HapMap chromosome {chrom}: {len(hapmap_chr)} SNPs")
        
        # 读取1000G数据
        kg_chr = read_1000g_snps(chrom, kg_base_path)
        if kg_chr.empty:
            continue
        logger.info(f"1000G chromosome {chrom}: {len(kg_chr)} SNPs")
        
        # 计数器
        matched_by_id = 0
        matched_by_pos = 0
        matched_total = set()
        
        # 通过rs ID匹配
        common_ids = set(hapmap_chr.index) & set(kg_chr.index)
        for snp_id in common_ids:
            if check_allele_match(hapmap_chr.loc[snp_id], kg_chr.loc[snp_id]):
                matched_by_id += 1
                matched_total.add(snp_id)
        
        # 通过位置匹配（对于ID不匹配的SNP）
        hapmap_pos_dict = {(row['position'], row['ref'], row['alt']): idx 
                          for idx, row in hapmap_chr.iterrows() 
                          if idx not in matched_total}
                          
        for idx, row in kg_chr.iterrows():
            if idx not in matched_total:
                pos_key = (row['position'], row['ref'], row['alt'])
                if pos_key in hapmap_pos_dict:
                    matched_by_pos += 1
                    matched_total.add(hapmap_pos_dict[pos_key])
        
        results.append({
            'chromosome': chrom,
            'hapmap_snps': len(hapmap_chr),
            '1000g_snps': len(kg_chr),
            'id_matches': matched_by_id,
            'pos_matches': matched_by_pos,
            'total_matches': len(matched_total),
            'hapmap_overlap': len(matched_total) / len(hapmap_chr) * 100 if len(hapmap_chr) > 0 else 0,
            '1000g_overlap': len(matched_total) / len(kg_chr) * 100 if len(kg_chr) > 0 else 0
        })
        
        # 输出一些不匹配的例子
        if len(results) == 1:  # 只对第一条染色体做详细分析
            unmatched_hapmap = set(hapmap_chr.index) - matched_total
            logger.info("\nExample unmatched HapMap SNPs:")
            for snp_id in list(unmatched_hapmap)[:5]:
                snp = hapmap_chr.loc[snp_id]
                logger.info(f"{snp_id}: pos={snp['position']}, {snp['ref']}/{snp['alt']}")
    
    # 创建汇总DataFrame
    summary_df = pd.DataFrame(results)
    summary_df.set_index('chromosome', inplace=True)
    
    # 添加总计行
    total_dict = {
        'hapmap_snps': summary_df['hapmap_snps'].sum(),
        '1000g_snps': summary_df['1000g_snps'].sum(),
        'id_matches': summary_df['id_matches'].sum(),
        'pos_matches': summary_df['pos_matches'].sum(),
        'total_matches': summary_df['total_matches'].sum()
    }
    total_dict['hapmap_overlap'] = total_dict['total_matches'] / total_dict['hapmap_snps'] * 100
    total_dict['1000g_overlap'] = total_dict['total_matches'] / total_dict['1000g_snps'] * 100
    
    summary_df.loc['Total'] = pd.Series(total_dict)
    
    return summary_df

def main():
    print("\nStarting SNP dataset comparison...")
    summary_df = compare_snp_sets()
    
    print("\nComparison Summary:")
    print("=" * 80)
    print(summary_df)
    
    # 保存结果
    summary_df.to_csv('snp_comparison_summary.csv')
    print("\nResults saved to snp_comparison_summary.csv")

if __name__ == "__main__":
    main()

INFO:__main__:
Processing chromosome 1
INFO:__main__:HapMap chromosome 1: 55983 SNPs



Starting SNP dataset comparison...


INFO:__main__:
Processing chromosome 2
INFO:__main__:HapMap chromosome 2: 58409 SNPs
INFO:__main__:
Processing chromosome 3
INFO:__main__:HapMap chromosome 3: 48655 SNPs
INFO:__main__:
Processing chromosome 4
INFO:__main__:HapMap chromosome 4: 43809 SNPs
INFO:__main__:
Processing chromosome 5
INFO:__main__:HapMap chromosome 5: 45045 SNPs
INFO:__main__:
Processing chromosome 6
INFO:__main__:HapMap chromosome 6: 46016 SNPs
INFO:__main__:
Processing chromosome 7
INFO:__main__:HapMap chromosome 7: 38578 SNPs
INFO:__main__:
Processing chromosome 8
INFO:__main__:HapMap chromosome 8: 38291 SNPs
INFO:__main__:
Processing chromosome 9
INFO:__main__:HapMap chromosome 9: 32252 SNPs
INFO:__main__:
Processing chromosome 10
INFO:__main__:HapMap chromosome 10: 36847 SNPs
INFO:__main__:
Processing chromosome 11
INFO:__main__:HapMap chromosome 11: 36514 SNPs
INFO:__main__:
Processing chromosome 12
INFO:__main__:HapMap chromosome 12: 33528 SNPs
INFO:__main__:
Processing chromosome 13
INFO:__main__:HapMa

KeyError: "None of ['chromosome'] are in the columns"

In [ ]:
debug_snp_format()

In [41]:
from CB_02704 import *
import pandas as pd
import numpy as np
from typing import Dict, Set, Tuple
import subprocess
import os

def get_1000g_positions(vcf_file: str) -> Set[str]:
    """Get SNP positions from filtered 1000G VCF file"""
    cmd = f"bcftools query -f '%CHROM:%POS\n' {vcf_file}"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return set(result.stdout.strip().split('\n'))

def get_all_1000g_positions() -> Set[str]:
    """Get all SNP positions from filtered 1000G VCF files"""
    all_positions = set()
    for chr in range(1, 23):
        vcf_file = f"../data/common_snps/common_snps.chr{chr}.vcf.gz"
        positions = get_1000g_positions(vcf_file)
        all_positions.update(positions)
    return all_positions

def filter_hapmap3(population: str, kg_positions: Set[str], data_dir: str) -> Tuple[np.ndarray, pd.DataFrame]:
    """Filter HapMap3 data for a population to keep only common SNPs"""
    # Read HapMap3 data
    print(f"Reading {population} data...")
    snp_data = read_snp_pop(population)
    geno_data = read_geno_pop(population)
    ind_data = read_ind_pop(population)
    
    # Create position strings for comparison
    snp_positions = set(f"{row['chromosome']}:{row['position']}" 
                       for _, row in snp_data.iterrows())
    
    # Find common positions
    common_positions = snp_positions.intersection(kg_positions)
    
    # Get indices of SNPs to keep
    keep_idx = [i for i, (_, row) in enumerate(snp_data.iterrows())
                if f"{row['chromosome']}:{row['position']}" in common_positions]
    
    # Filter data
    filtered_geno = geno_data[keep_idx]
    filtered_snps = snp_data.iloc[keep_idx]
    
    # Print statistics
    print(f"\nFiltering statistics for {population}:")
    print(f"Original SNPs: {len(snp_data):,}")
    print(f"Retained SNPs: {len(filtered_snps):,}")
    print(f"Retention rate: {(len(filtered_snps)/len(snp_data)*100):.2f}%")
    
    return filtered_geno, filtered_snps, ind_data

def save_hapmap3_files(population: str, geno_data: np.ndarray, 
                      snp_data: pd.DataFrame, ind_data: pd.DataFrame,
                      output_dir: str):
    """Save filtered HapMap3 data in original format"""
    os.makedirs(output_dir, exist_ok=True)
    
    # Save .geno file
    geno_path = os.path.join(output_dir, f"{population}.geno")
    with open(geno_path, 'w') as f:
        for row in geno_data:
            f.write(''.join(map(str, row)) + '\n')
    
    # Save .snp file
    snp_path = os.path.join(output_dir, f"{population}.snp")
    snp_data.to_csv(snp_path, sep='\t', index=True, header=False)
    
    # Save .ind file (unchanged)
    ind_path = os.path.join(output_dir, f"{population}.ind")
    ind_data.to_csv(ind_path, sep='\t', index=True, header=False)
    
    print(f"\nSaved filtered files for {population}:")
    print(f"Genotype data: {geno_path}")
    print(f"SNP data: {snp_path}")
    print(f"Individual data: {ind_path}")

def main():
    # Configuration
    DATA_DIR = "/Users/jingl1/Desktop/CMU/02704/hw_data/02704_data"
    OUTPUT_DIR = "filtered_hapmap3"
    POPULATIONS = ['ASW', 'CEU', 'CHB', 'JPT', 'LWK', 'MXL', 'TSI', 'YRI']
    
    # Get all 1000G positions
    print("Getting 1000G SNP positions...")
    kg_positions = get_all_1000g_positions()
    print(f"Found {len(kg_positions):,} SNPs in filtered 1000G data")
    
    # Process each population
    for population in POPULATIONS:
        print(f"\nProcessing {population}...")
        
        # Filter population data
        filtered_geno, filtered_snps, ind_data = filter_hapmap3(
            population, kg_positions, DATA_DIR)
        
        # Save filtered data
        save_hapmap3_files(population, filtered_geno, filtered_snps, 
                          ind_data, OUTPUT_DIR)

if __name__ == "__main__":
    main()

In [30]:
from CB_02704 import *
import pandas as pd
import numpy as np
from typing import Dict
from pathlib import Path

def read_hapmap_data(pop: str, data_dir: str) -> pd.DataFrame:
    """Read HapMap3 SNP data"""
    snp_file = Path(data_dir) / f"{pop}.snp"
    return pd.read_csv(snp_file, sep='\s+', header=None,
                      names=['SNP', 'chromosome', 'morgans', 'position', 'ref', 'alt'])

def read_1000g_data(pop: str, data_dir: str) -> pd.DataFrame:
    """Read 1000G SNP data"""
    snp_file = Path(data_dir) / f"{pop}.snp"
    return pd.read_csv(snp_file, sep='\s+', header=None,
                      names=['chromosome', 'position', 'ref', 'alt', 'morgans'])

def check_allele_match(row1, row2) -> bool:
    """Check if alleles match considering possible flips"""
    return ((row1['ref'] == row2['ref'] and row1['alt'] == row2['alt']) or
            (row1['ref'] == row2['alt'] and row1['alt'] == row2['ref']))

def verify_population_alignment(pop: str, 
                              hapmap_dir: str="../data/filtered_hapmap3",
                              kg_dir: str="../data/1000g/populations") -> Dict:
    """Verify SNP alignment between HapMap3 and 1000G for a population"""
    
    # Read both datasets
    print(f"Reading data for {pop}...")
    hapmap_snps = read_hapmap_data(pop, hapmap_dir)
    kg_snps = read_1000g_data(pop, kg_dir)
    
    # Create position identifiers
    hapmap_pos = set(zip(hapmap_snps['chromosome'], hapmap_snps['position']))
    kg_pos = set(zip(kg_snps['chromosome'], kg_snps['position']))
    
    # Find common positions
    common_positions = hapmap_pos.intersection(kg_pos)
    
    # Check allele matching for common SNPs
    if common_positions:
        match_count = 0
        total_checked = 0
        
        for chrom, pos in common_positions:
            hm_row = hapmap_snps[(hapmap_snps['chromosome'] == chrom) & 
                                (hapmap_snps['position'] == pos)].iloc[0]
            kg_row = kg_snps[(kg_snps['chromosome'] == chrom) & 
                            (kg_snps['position'] == pos)].iloc[0]
            
            if check_allele_match(hm_row, kg_row):
                match_count += 1
            total_checked += 1
        
        allele_match_rate = (match_count / total_checked * 100) if total_checked > 0 else 0
    else:
        allele_match_rate = 0
    
    return {
        'Population': pop,
        'HapMap3_SNPs': len(hapmap_snps),
        'KG_SNPs': len(kg_snps),
        'Common_SNPs': len(common_positions),
        'Unique_to_HapMap3': len(hapmap_pos - kg_pos),
        'Unique_to_KG': len(kg_pos - hapmap_pos),
        'Allele_Match_Rate': allele_match_rate
    }

def main():
    """Verify alignment for all populations"""
    populations = ['ASW', 'CEU', 'CHB', 'JPT', 'LWK', 'MXL', 'TSI', 'YRI']
    
    # Verify each population
    all_stats = []
    for pop in populations:
        stats = verify_population_alignment(pop)
        all_stats.append(stats)
        
        # Print detailed results for each population
        print(f"\nResults for {pop}:")
        for key, value in stats.items():
            if isinstance(value, (int, float)):
                print(f"{key}: {value:,}" if isinstance(value, int) else f"{key}: {value:.2f}%")
            else:
                print(f"{key}: {value}")
    
    # Create summary DataFrame
    summary_df = pd.DataFrame(all_stats)
    
    # Add percentage of common SNPs
    summary_df['Common_SNP_Pct'] = (summary_df['Common_SNPs'] / summary_df['HapMap3_SNPs'] * 100).round(2)
    
    # Display and save summary
    print("\nOverall Summary:")
    print(summary_df.to_string(index=False))
    summary_df.to_csv("dataset_alignment_summary.csv", index=False)
    
    return summary_df

if __name__ == "__main__":
    summary_df = main()

Reading data for ASW...

Results for ASW:
Population: ASW
HapMap3_SNPs: 21,694
KG_SNPs: 21,664
Common_SNPs: 21,643
Unique_to_HapMap3: 0
Unique_to_KG: 0
Allele_Match_Rate: 26.58%
Reading data for CEU...

Results for CEU:
Population: CEU
HapMap3_SNPs: 21,694
KG_SNPs: 21,664
Common_SNPs: 21,643
Unique_to_HapMap3: 0
Unique_to_KG: 0
Allele_Match_Rate: 26.58%
Reading data for CHB...

Results for CHB:
Population: CHB
HapMap3_SNPs: 21,694
KG_SNPs: 21,664
Common_SNPs: 21,643
Unique_to_HapMap3: 0
Unique_to_KG: 0
Allele_Match_Rate: 26.58%
Reading data for JPT...

Results for JPT:
Population: JPT
HapMap3_SNPs: 21,694
KG_SNPs: 21,664
Common_SNPs: 21,643
Unique_to_HapMap3: 0
Unique_to_KG: 0
Allele_Match_Rate: 26.58%
Reading data for LWK...

Results for LWK:
Population: LWK
HapMap3_SNPs: 21,694
KG_SNPs: 21,664
Common_SNPs: 21,643
Unique_to_HapMap3: 0
Unique_to_KG: 0
Allele_Match_Rate: 26.58%
Reading data for MXL...

Results for MXL:
Population: MXL
HapMap3_SNPs: 21,694
KG_SNPs: 21,664
Common_SNPs: 21

In [42]:
# 初始化比较器
comparator = SNPDatasetComparison()

# 加载数据
comparator.load_hapmap_data("hapmap3")
comparator.load_1000g_data("path_to_1000g_data")

# 执行比较分析
results, chr_analysis = comparator.compare_snp_overlap()
common_snps = set(comparator.hapmap_data['snp'].index).intersection(
    set(comparator.kg_data['snp'].index))
allele_analysis = comparator.analyze_allele_consistency(common_snps)

# 生成报告
report = generate_report(results, chr_analysis, allele_analysis)
print(report)

TypeError: SNPDatasetComparison.load_hapmap_data() missing 1 required positional argument: 'ind_file'

In [39]:
from CB_02704 import *
import pandas as pd
import numpy as np
from typing import Dict, Set, List, Tuple
import subprocess
import os

def get_1000g_positions(vcf_file: str) -> Set[str]:
    """Extract SNP positions from filtered 1000G VCF file"""
    cmd = f"bcftools query -f '%CHROM\t%POS\n' {vcf_file}"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    # Create position IDs in the same format as HapMap3
    positions = set()
    for line in result.stdout.strip().split('\n'):
        if line:  # Skip empty lines
            chrom, pos = line.split('\t')
            positions.add(f"{chrom}:{pos}")
    return positions

def get_all_1000g_positions(common_snps_dir: str) -> Set[str]:
    """Get SNP positions from all chromosomes in filtered 1000G data"""
    all_positions = set()
    for chrom in range(1, 23):
        vcf_file = f"{common_snps_dir}/common_snps.chr{chrom}.vcf.gz"
        if os.path.exists(vcf_file):
            print(f"Reading chromosome {chrom}...")
            positions = get_1000g_positions(vcf_file)
            all_positions.update(positions)
            print(f"Found {len(positions):,} SNPs in chromosome {chrom}")
    return all_positions

def filter_hapmap_data(pop: str, common_positions: Set[str]) -> Tuple[np.ndarray, pd.DataFrame, pd.DataFrame]:
    """Filter HapMap3 population data to keep only common SNPs"""
    print(f"Reading {pop} data...")
    
    # Read population data
    geno_data = read_geno_pop(pop)
    snp_data = read_snp_pop(pop)
    ind_data = read_ind_pop(pop)
    
    # Create position identifiers for HapMap SNPs
    snp_data['pos_id'] = snp_data['chromosome'].astype(str) + ':' + snp_data['position'].astype(str)
    
    # Find indices of common SNPs
    keep_idx = snp_data['pos_id'].isin(common_positions)
    
    # Filter data
    filtered_geno = geno_data[keep_idx]
    filtered_snps = snp_data[keep_idx].drop('pos_id', axis=1)
    
    # Print filtering statistics
    print(f"\nFiltering statistics for {pop}:")
    print(f"Original SNPs: {len(snp_data):,}")
    print(f"Retained SNPs: {len(filtered_snps):,}")
    print(f"Retention rate: {(len(filtered_snps)/len(snp_data)*100):.2f}%\n")
    
    return filtered_geno, filtered_snps, ind_data

def save_hapmap_format(geno_data: np.ndarray, snp_data: pd.DataFrame, 
                      ind_data: pd.DataFrame, pop: str, output_dir: str):
    """Save filtered data in HapMap3 format"""
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Save genotype data (.geno)
    geno_file = f"{output_dir}/{pop}.geno"
    with open(geno_file, 'w') as f:
        for row in geno_data:
            # Convert masked array to regular array and handle masked values
            row_data = row.filled(9)  # Fill masked values with 9
            f.write(''.join(map(str, row_data)) + '\n')
    
    # Save SNP data (.snp)
    snp_file = f"{output_dir}/{pop}.snp"
    snp_data.to_csv(snp_file, sep='\t', index=True, header=False)
    
    # Save individual data (.ind)
    ind_file = f"{output_dir}/{pop}.ind"
    ind_data.to_csv(ind_file, sep='\t', index=True, header=False)
    
    print(f"Saved filtered files for {pop}:")
    print(f"- Genotype data: {geno_file}")
    print(f"- SNP data: {snp_file}")
    print(f"- Individual data: {ind_file}\n")

def main():
    # Configuration
    KG_DIR = '../data/common_snps'  # Directory with filtered 1000G VCF files
    OUTPUT_DIR = '../data/filtered_hapmap3'  # Output directory for filtered HapMap3 data
    
    # Get positions from all chromosomes
    print("Getting 1000G SNP positions from all chromosomes...")
    common_positions = get_all_1000g_positions(KG_DIR)
    print(f"Found total of {len(common_positions):,} SNPs in filtered 1000G data\n")
    
    # HapMap3 populations to process
    populations = ['ASW', 'CEU', 'CHB', 'JPT', 'LWK', 'MXL', 'TSI', 'YRI']
    
    # Process each population
    stats = []
    for pop in populations:
        # Filter population data
        geno_filtered, snp_filtered, ind_data = filter_hapmap_data(pop, common_positions)
        
        # Save filtered data
        save_hapmap_format(geno_filtered, snp_filtered, ind_data, pop, OUTPUT_DIR)
        
        # Collect statistics
        stats.append({
            'Population': pop,
            'Original_SNPs': len(read_snp_pop(pop)),
            'Filtered_SNPs': len(snp_filtered),
            'Individuals': len(ind_data),
            'Retention_Rate': (len(snp_filtered)/len(read_snp_pop(pop))*100)
        })
    
    # Create summary report
    stats_df = pd.DataFrame(stats)
    summary_file = f"{OUTPUT_DIR}/filtering_summary.csv"
    stats_df.to_csv(summary_file, index=False)
    print(f"Saved summary report to: {summary_file}")
    
    return stats_df

if __name__ == "__main__":
    main()

Getting 1000G SNP positions from all chromosomes...
Reading chromosome 1...
Found 1,616 SNPs in chromosome 1
Reading chromosome 2...
Found 1,784 SNPs in chromosome 2
Reading chromosome 3...
Found 1,445 SNPs in chromosome 3
Reading chromosome 4...
Found 1,366 SNPs in chromosome 4
Reading chromosome 5...
Found 1,323 SNPs in chromosome 5
Reading chromosome 6...
Found 1,421 SNPs in chromosome 6
Reading chromosome 7...
Found 1,211 SNPs in chromosome 7
Reading chromosome 8...
Found 1,381 SNPs in chromosome 8
Reading chromosome 9...
Found 1,036 SNPs in chromosome 9
Reading chromosome 10...
Found 1,126 SNPs in chromosome 10
Reading chromosome 11...
Found 1,153 SNPs in chromosome 11
Reading chromosome 12...
Found 1,045 SNPs in chromosome 12
Reading chromosome 13...
Found 772 SNPs in chromosome 13
Reading chromosome 14...
Found 666 SNPs in chromosome 14
Reading chromosome 15...
Found 637 SNPs in chromosome 15
Reading chromosome 16...
Found 835 SNPs in chromosome 16
Reading chromosome 17...
Found

In [35]:
from CB_02704 import *
import pandas as pd
from pathlib import Path

def inspect_files_and_paths(pop='ASW'):
    """Inspect the actual file paths and first few lines of data"""
    # Define paths
    hapmap_dir = Path("../data/filtered_hapmap3")
    kg_dir = Path("../data/1000g/populations")
    
    hapmap_file = hapmap_dir / f"{pop}.snp"
    kg_file = kg_dir / f"{pop}.snp"
    
    print(f"Checking paths for population {pop}:")
    print(f"HapMap3 file: {hapmap_file.absolute()}")
    print(f"1000G file: {kg_file.absolute()}")
    
    print(f"\nChecking file existence:")
    print(f"HapMap3 file exists: {hapmap_file.exists()}")
    print(f"1000G file exists: {kg_file.exists()}")
    
    print("\nDirectory contents:")
    print("\nHapMap3 directory:")
    print([x.name for x in hapmap_dir.glob("*.snp")])
    
    print("\n1000G directory:")
    print([x.name for x in kg_dir.glob("*.snp")])
    
    if hapmap_file.exists() and kg_file.exists():
        print(f"\nFirst 3 lines from {pop} HapMap3 file:")
        with open(hapmap_file) as f:
            for i, line in enumerate(f):
                if i < 3:
                    print(line.strip())
        
        print(f"\nFirst 3 lines from {pop} 1000G file:")
        with open(kg_file) as f:
            for i, line in enumerate(f):
                if i < 3:
                    print(line.strip())

if __name__ == "__main__":
    populations = ['ASW', 'CEU', 'CHB', 'JPT', 'LWK', 'MXL', 'TSI', 'YRI']
    for pop in populations:
        print("="*50)
        inspect_files_and_paths(pop)
        print("\n")

Checking paths for population ASW:
HapMap3 file: /Users/jingl1/Documents/GitHub/02704_Cross-Database-Analysis-of-Population-Structure/scripts/../data/filtered_hapmap3/ASW.snp
1000G file: /Users/jingl1/Documents/GitHub/02704_Cross-Database-Analysis-of-Population-Structure/scripts/../data/1000g/populations/ASW.snp

Checking file existence:
HapMap3 file exists: True
1000G file exists: True

Directory contents:

HapMap3 directory:
['TSI.snp', 'ASW.snp', 'JPT.snp', 'MXL.snp', 'YRI.snp', 'LWK.snp', 'CEU.snp', 'CHB.snp']

1000G directory:
['TSI.snp', 'ASW.snp', 'JPT.snp', 'MXL.snp', 'YRI.snp', 'LWK.snp', 'CEU.snp', 'CHB.snp']

First 3 lines from ASW HapMap3 file:
rs4442317	1	0.0	1096647	T	C
rs6697886	1	0.0	1163474	G	A
rs11260577	1	0.0	1200334	C	T

First 3 lines from ASW 1000G file:
1	1096647	C	T	0.0
1	1163474	A	G	0.0
1	1200334	T	C	0.0


Checking paths for population CEU:
HapMap3 file: /Users/jingl1/Documents/GitHub/02704_Cross-Database-Analysis-of-Population-Structure/scripts/../data/filtered

In [ ]:
vcf_file = "../data/common_snps/common_snps.chr15.vcf.gz"
# Check if file exists
print(f"File exists: {os.path.exists(vcf_file)}")

# Try to get first few lines with bcftools
cmd = f"bcftools view {vcf_file} | head"
result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
print("\nFirst few lines:")
print(result.stdout)